Продолжаем знакомство с моделями МО в области обучения с учителем. На этот раз поговорим о задаче **классификации**. Вспомним, где находится классификация на нашей карте машинного обучения:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-3_2 (1).png)

Вначале мы снова обратимся к классу линейных моделей и рассмотрим **логистическую регрессию**.

↓

Затем поговорим о **деревьях решений** для задачи классификации и научимся строить из этих деревьев целый лес.

**Цели данного модуля:**

* Познакомиться с принципами работы модели логистической регрессии для решения задачи классификации. 
* Рассмотреть метрики классификации и научиться оценивать качество моделей, решающих данную задачу.
* Узнать принципы построения деревьев решений.
* Изучить основы ансамблевых моделей типа бэггинг на примере случайного леса.
* Научиться применять деревья решений и случайные леса для решения задачи классификации.

`✍ Ранее мы обсуждали модель линейной регрессии, которая предназначена для решения задачи регрессии. Теперь нам предстоит разобраться с тем, как преобразовать данную модель, чтобы она решала задачу классификации.`

Для начала вспомним, что такое классификация.

**Задача классификации (classification)** *— задача, в которой мы пытаемся предсказать класс объекта на основе признаков в наборе данных. То есть задача сводится к предсказанию целевого признака, который является **категориальным**.*

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-3_9.png)

Когда классов, которые мы хотим предсказать, только два, классификация называется **бинарной**. Например, мы можем предсказать, болен ли пациент раком, является ли изображение человеческим лицом, является ли письмо спамом и т. д.

Когда классов, которые мы хотим предсказать, более двух, классификация называется **мультиклассовой (многоклассовой)**. Например, предсказание модели самолёта по радиолокационным снимкам, классификация животных на фотографиях, определение языка, на котором говорит пользователь, разделение писем на группы.

    → Для простоты мы пока разберёмся с бинарной классификацией, а в следующем юните обобщим результат на мультиклассовую.

Что вообще означает «решить задачу классификации»? Это значит построить разделяющую поверхность в пространстве признаков, которая делит пространство на части, каждая из которых соответствует определённому классу. 

Ниже представлены примеры разделяющих поверхностей, которые производят бинарную классификацию. Красным и синим цветом обозначены классы, зелёным — собственно поверхность, которая делит пространство признаков на две части. В каждой из этих частей находятся только наблюдения определённого класса.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_1.png)

*Модели, которые решают задачу классификации, называются **классификаторами (classifier)**.*

Если взять в качестве разделяющей поверхности некоторую плоскость (ровная поверхность на первом рисунке), то мы получаем модель логистической регрессии, которая тесно связана с рассмотренной нами ранее линейной регрессией.

Давайте для начала вспомним, как выглядит уравнение модели линейной регрессии в общем случае:

![](./image/2024-08-01_20-59-15.png)

В общем случае это уравнение гиперплоскости, которая стремится приблизить зависимость целевой переменной от $m$ факторов.

* Когда фактор всего один, уравнение задаёт прямую:

$y^` = w_0 + w_1x$

* Когда факторов два, уравнение задаёт плоскость:

$y^`=w_0+w_1x_1+w_2x_2$

→ Но всё это работает только в том случае, когда целевой признак $y$, который мы хотим предсказать, является числовым, например цена, вес, время аренды и т. д.

Что же делать с этой моделью, когда целевой признак $y$ является категориальным? Например, является письмо спамом или обычным письмом?

Можно предположить, что, раз у нас есть две категории, мы можем обозначить категории за $y=1$ (Спам) и $y=0$ (Не спам) и обучить линейную регрессию предсказывать 0 и 1.

Но результат будет очень плохим. Выглядеть это будет примерно так:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_2.png)

Для больших значений $x$ прямая будет выдавать значения больше 1, а для очень маленьких — меньше 0. Что это значит? Непонятно. Непонятно и то, что делать со значениями в диапазоне от 0 до 1. Да, можно относить значения на прямой выше 0.5 к классу 1, а меньше либо равным 0.5 — к классу 0, но это всё «костыли».

`Идея! Давайте переведём задачу классификации в задачу регрессии. Вместо предсказания класса будем предсказывать вероятность принадлежности к этому классу.` 

Модель должна выдавать некоторую вероятность $P$, которая будет определять, принадлежит ли данный объект к классу 1: например, вероятность того, что письмо является спамом. При этом вероятность того, что письмо является обычным письмом (класс 0), определяется как $Q=1-P$.  

Когда модель будет обучена на предсказание вероятности, мы зададим некоторый порог вероятности. Если предсказанная вероятность будет выше этого порога, мы определим объект к классу 1, а если ниже — к классу 0.

Например, стандартный порог равен 0.5. То есть если вероятность $P>0.5$, мы будем считать письмо спамом, а если $P<=0.5$ — обычным информативным письмом.

В итоге мы добьёмся того, что будем предсказывать не дискретный категориальный, а непрерывный числовой признак, который лежит в диапазоне [0, 1]. А это уже знакомая нам задача регрессии.

    → Однако остался главный вопрос: как научить модель предсказывать вероятности, ведь они должны лежать строго в диапазоне от 0 до 1, а предсказания линейной регрессии лежат в диапазоне от $-infinity$ до $+infinity$? 

Тут-то мы и приходим к модели логистической регрессии — **регрессии вероятностей**.

### <center> ОБЩЕЕ ПРЕДСТАВЛЕНИЕ О ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

**Логистическая регрессия (Logistic Regression)** *— одна из простейших моделей для решения задачи классификации. Несмотря на простоту, модель входит в топ часто используемых алгоритмов классификации в Data Science.*

В основе логистической регрессии лежит логистическая функция (*logistic function*) $sigma(z)$ — отсюда и название модели. Однако более распространённое название этой функции — **сигмόида (sigmoid)**. Записывается она следующим образом:

![](./image/2024-08-01_21-07-45.png)

Примечание. Здесь $e$ — экспонента или число Эйлера. Это число является бесконечным, а его значение обычно принимают равным $2.718...$.

А вот график зависимости сигмоиды от аргумента $z$:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_3.png)

**В чём преимущество этой функции?**

*У сигмоиды есть два очень важных для нас свойства:*

* Значения сигмоиды $sigma(z)$ лежат в диапазоне от 0 до 1 при любых значения аргумента $z$: какой бы $z$ вы ни подставили, число меньше 0 или больше 1 вы не получите.
* Сигмоида выдаёт значения $sigma(z)>0.5$ при её аргументе $x>0$, $sigma(z)<0.5$ — при $z<0 и $sigma(z)=0.5$ — при $z=0$.

Это ведь и есть свойства вероятности! Выходом сигмоиды является число от 0 до 1, которое можно интерпретировать как вероятность принадлежности к классу 1. Её мы и пытаемся предсказать.

**Основная идея** модели логистической регрессии: возьмём модель линейной регрессии (обозначим её выход за $z$)

![](./image/2024-08-01_21-17-21.png)

и подставим выход модели $z$ в функцию сигмоиды, чтобы получить искомые оценки вероятности (в математике принято писать оценочные величины с «шапкой» наверху, а истинные значения — без «шапки», это чистая формальность):

![](./image/2024-08-01_21-18-54.png)

**Примечание.** *Далее в модуле мы будем называть оценки вероятности $P^`$ просто вероятностью, но только для краткости. Это не значит, что эти оценки являются истинными вероятностями принадлежности к каждому из классов (их нельзя сосчитать, так как для этого нужна выборка бесконечного объёма). Если вы употребляете термин «вероятности» на собеседованиях, обязательно предварительно укажите, что вы подразумеваете оценку вероятности.*

Обучать будем всё в совокупности, пытаясь получить наилучшую оценку вероятности $P^`$. Если вероятность $P^`>0.5$, относим объект к классу 1, а если $P^`<=0.5$, относим объект к классу 0. 

Математически это записывается следующей формулой:

![](./image/2024-08-01_21-21-56.png)

**Примечание.** *В данном выражении $I[P^`]$ называется **индикаторной функцией**. Она возвращает 1, если её значение больше какого-то порога, и 0 — в противном случае. Математики часто записывают просто квадратные скобки, опуская символ $I$: $[P^`]$.*

**Чего мы добились таким преобразованием?**

Если мы обучим модель, то есть подберём  коэффициенты $w_0, w_1, w_2, ..., w_m$ (как их найти, обсудим чуть позже) таким образом, что для объектов класса 1 модель линейной регрессии начнёт выдавать положительное число, а для класса 0 — выдавать отрицательное число, то тогда, подставив предсказание линейной регрессии $z$ в сигмоиду, мы сможем получать вероятности принадлежности к каждому из классов в диапазоне от 0 до 1.

Далее по порогу вероятности мы сможем определять, к какому классу принадлежит объект.

Это и есть наша цель. Мы свели задачу классификации к задаче регрессии для предсказания вероятностей. 

Для бинарной классификации описанное выше будет выглядеть следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_4.png)

![](./image/2024-08-01_21-26-21.png)

**Разберёмся с геометрией**

Возьмём частный случай, когда класс объекта зависит от двух признаков — $x_1$ и $x_2$.

![](./image/2024-08-01_21-31-25.png)

![](./image/2024-08-01_21-32-32.png)

**В чём математический секрет?**

Математически подстановка в уравнение плоскости точки, которая не принадлежит ей (находится ниже или выше), означает вычисление расстояния от этой точки до плоскости.

* Если точка находится ниже плоскости, расстояние будет отрицательным ($z<0$).
* Если точка находится выше плоскости, расстояние будет положительным ($z>0$).
* Если точка находится на самой плоскости, $z=0$.

Мы знаем, что подстановка отрицательных чисел в сигмоиду приведёт к вероятности $P^`<0.5$, а постановка положительных — к вероятности $P^`>0.5$. 

*Таким образом, ключевым моментом в предсказании логистической регрессии является расстояние от точки до разделяющей плоскости в пространстве факторов. Это расстояние в литературе часто называется* **отступом (margin)**. 

В этом и состоит секрет работы логистической регрессии.

*Чем больше расстояние от точки, находящейся выше разделяющей плоскости, до самой плоскости, тем больше оценка вероятности принадлежности к классу 1.*

Попробуйте подставить различные координаты точек в модель логистической регрессии и убедитесь в этом.

Можно построить тепловую карту, которая показывает, чему равны вероятности в каждой точке пространства:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_7.png)

`На рисунке точки, которые относятся к классу непоступивших абитуриентов, лежащие ниже разделяющей плоскости, находятся в красной зоне. Чем насыщеннее красный цвет, тем ниже вероятность того, что абитуриент поступит в аспирантуру. И наоборот, точки, которые относятся к классу поступивших абитуриентов, лежащие выше разделяющей плоскости, находятся в синей зоне. Чем насыщеннее синий цвет, тем выше вероятность того, что абитуриент поступит в аспирантуру.`

Для случая зависимости целевого признака $y$ от трёх факторов $x_1$, $x_2$ и $x_3$, например от баллов за два экзамена и рейтинга университета, из которого выпустился абитуриент, выражение для $z$ будет иметь вид:

$z=w_0+w_1x_1+w_2x_2+w_3x_3$

Уравнение задаёт плоскость в четырёхмерном пространстве. Но если вспомнить, что $y$ — категориальный признак и классы можно обозначить цветом, то получится перейти в трёхмерное пространство. Разделяющая плоскость будет выглядеть следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_8.png)

В общем случае, когда у нас есть зависимость от $m$ факторов, линейное выражение, находящееся под сигмоидой, будет обозначать **разделяющую гиперплоскость**.

![](./image/2024-08-01_21-39-41.png)

### <center> ПОИСК ПАРАМЕТРОВ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

Итак, мы разобрались с тем, как выглядит модель логистической регрессии и что она означает в геометрическом смысле.

Но остался один главный вопрос: как найти такие коэффициенты $w=(w_0, w_1, w_2, ..., w_m)$, чтобы гиперплоскость разделяла пространство наилучшим образом?

Вновь обратимся к нашей схеме минимизации эмпирического риска:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml1-2_1.png)

Можно предположить, что стоит использовать метод наименьших квадратов. Введём функцию ошибки — средний квадрат разности MSE между истинными классами $y$ и предсказанными классами $y^`$ и попытаемся его минимизировать.

Сразу можно достоверно предсказать, что результат такого решения будет плохим, поэтому воздержимся от его использования.

Здесь нужен другой подход. Это **метод максимального правдоподобия (Maximum Likelihood Estimation — MLE)**. 

**Правдоподобие** *— это оценка того, насколько вероятно получить истинное значение целевой переменной  при данных  и параметрах $w$.*

Данный метод позволяет получить функцию правдоподобия.

**Цель метода** *— найти такие параметры $w=(w_0, w_1, w_2, ..., w_m)$, в которых наблюдается максимум функции правдоподобия.*

А мы пока что опустим математические детали метода и приведём только конечную формулу:

![](./image/2024-08-01_21-45-58.png)

Не пугайтесь. Давайте разберёмся, что есть что и как работает эта функция.

* $n$ — количество наблюдений.
* $y_i$ — это истинный класс (1 или 0) для -ого объекта из набора данных.
* $P^`=sigma(z_i)$ — предсказанная с помощью логистической регрессии вероятность принадлежности к классу 1 для $i$-ого объекта из набора данных.
* $z_i$ — результат подстановки -ого объекта из набора данных в уравнение разделяющей плоскости $z_i=w^`*x_i^`$.
* $log$ — логарифм (обычно используется натуральный логарифм по основанию $e-ln$).

![](./image/2024-08-01_21-54-39.png)

Такие расчёты можно производить для любых значений параметров, меняется только оценка вероятности $P_i^`$.

**Примечание.** *К сожалению, функция likelihood не имеет интерпретации, то есть нельзя сказать, что значит число -2.34 в контексте правдоподобия.*

Цель — найти такие параметры, при которых наблюдается максимум этой функции.

Теперь пора снова применить магию математики, чтобы привести задачу к привычному нам формату минимизации эмпирического риска. По правилам оптимизации, если поставить перед функцией минус, то задача оптимизации меняется на противоположную: был поиск максимума — станет поиском минимума.

Таким образом мы получим функцию потерь $L(w)$, которая носит название **«функция логистических потерь»**, или **logloss**. Также часто можно встретить название **кросс-энтропия**, или **cross-entropy loss**:

![](./image/2024-08-01_21-56-49.png)

Вот эту функцию мы и будем минимизировать в рамках поиска параметров логистической регрессии. Мы должны найти такие параметры разделяющей плоскости $w$, при которых наблюдается минимум logloss.

Знакомая задача? Всё то же самое, что и с линейной регрессией, только функция ошибки другая.

    → К сожалению, для такой функции потерь аналитическое решение оптимизационной задачи найти не получится: при расчётах получается, что его попросту не существует.

Но мы помним, что, помимо аналитических решений, есть и численные.

Например, для поиска параметров можно использовать знакомый нам градиентный спуск. Вспомним, как выглядит итерационная формула данного метода:

![](./image/2024-08-01_21-58-06.png)

Повторим её смысл: новое значение параметров $w^{(k+1)}$ получается путём сдвига текущих $w^{(k)}$ в сторону вектора антиградиента $Grad L(w^{(k)})$, умноженного на темп обучения $nu$.

    Мы уже знаем, что для того, чтобы повысить шанс пройти мимо локальных минимумов функции потерь, используется не сам градиентный спуск, а его модификации: например, можно использовать уже знакомый нам стохастический градиентный спуск (SGD).

`Помним, что применение градиентного спуска требует предварительного масштабирования данных (стандартизации/нормализации). В реализации логистической регрессии в sklearn предусмотрено ещё несколько методов оптимизации, для которых масштабирование не обязательно. О них мы упомянем в практической части модуля.`

Во избежание переобучения модели в функцию потерь логистической регрессии традиционно добавляется **регуляризация**. В реализации логистической регрессии в *sklearn* она немного отличается от той, что мы видели ранее для линейной регрессии.

При L1-регуляризации мы добавляем в функцию потерь $L(w)$ штраф из суммы модулей параметров, а саму функцию logloss умножаем на коэффициент $C$:

![](./image/2024-08-01_22-10-48.png)

Значение коэффициента $C$ — коэффициент, обратный коэффициенту регуляризации. Чем **больше** $C$, тем **меньше** «сила» регуляризации.

Предлагаем вам посмотреть на то, как будет меняться форма сигмоиды, разделяющей плоскости при минимизации функции потерь logloss (она обозначена как cross-entropy в виде концентрических кругов — вид сверху), с помощью обычного градиентного спуска (не стохастического) в виде анимации.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_12.gif)
![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_13.gif)
![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-2_14.gif)

Не волнуйтесь, все громоздкие формулы уже реализованы в классических библиотеках, таких как sklearn. Но нам важно понимать принцип того, что происходит «под капотом», чтобы верно интерпретировать результат и по возможности управлять им.

### <center> ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ В SKLEARN

    Мы будем работать со знакомым вам по модулю «Очистка данных» набором данных о диабете, первоначально полученным в Национальном институте диабета, болезней органов пищеварения и почек.

    Наша цель будет состоять в том, чтобы диагностически предсказать, есть ли у пациента диабет. На выбор экземпляров из более крупной базы данных было наложено несколько ограничений. В частности, все пациенты здесь — женщины не моложе 21 года из индейского племени Пима.

В модуле по очистке данных вы уже производили очистку этого набора данных:

* удалили дубликаты,
* удалили неинформативный признак Gender,
* обработали «скрытые» пропуски в данных,
* избавились от потенциальных выбросов.

#### Результат представлен в ./Логистическая_регрессия.ipynb

    ✍ Мы научились обучать модель логистической регрессии, чтобы решать с её помощью задачу классификации. Теперь мы познакомимся с метриками классификации, чтобы научиться оценивать качество модели.

Однако прежде чем мы перейдём к этим метрикам, нам очень важно освежить в памяти некоторую терминологию из статистики, а именно вспомнить, что такое ошибки I и II рода.

### <center>ОШИБКИ I И II РОДА С ТОЧКИ ЗРЕНИЯ КЛАССИФИКАЦИИ

Давайте рассмотрим предсказания алгоритма $y_i^`$ на конкретном объекте $x_i$ под номером из данных $i$ с точки зрения статистических гипотез.

Будем считать класс 1 (диабет есть) положительным исходом (Positive), а класс 0 (диабета нет) — отрицательным (Negative).

**Примечание.** *На первый взгляд такая терминология может показаться не совсем этичной, так как наличие диабета — это всё-таки отрицательный сценарий для пациента. Однако для унификации терминологии в машинном обучении в большинстве задач объекты класса 1 считаются объектами с наличием некоторого эффекта (болезнь есть / задолженность погашена / клиент ушёл / устройство отказало и т. д.), а объекты класса 0 — объектами с отсутствием этого эффекта (болезни нет / задолженность не погашена / клиент не ушёл / устройство работает без отказов и т. д.).*

Пусть у нас есть некоторый пациент $x_i$, и мы хотим понять, болен ли он диабетом. С точки зрения задачи классификации мы хотим предсказать истинный класс ($y_i$) пациента.

Нулевая гипотеза будет состоять в отсутствии эффекта (пациент не болен диабетом), то есть , а альтернативная — в его наличии (пациент болен диабетом) $y_i=0$, то есть $y_i=1$. В терминах статистических гипотез это будет записано так:

* $H_0$: Пациент $x_i$ не болеет диабетом $y_i=0$.
* $H_1$: Пациент $x_i$ болеет диабетом $y_i=1$.
Тогда у нас есть два случая, в которых мы можем допустить ошибку:

Ошибка I (первого) рода ($alpfa$-ошибка): отклонение нулевой гипотезы, когда она на самом деле верна, или **ложноположительный результат**. То есть мы предсказали, что пациент болен диабетом, хотя это не так.
Ошибка II (второго) рода ($betta$-ошибка): принятие нулевой гипотезы, когда она на самом деле ложна, или **ложноотрицательный результат**. То есть мы предсказали, что пациент здоров, хотя на самом деле он болен диабетом.

**Примечание.** *Как вы можете понять, в диагностических задачах для нас критичнее ошибка II рода. Последствия будут более серьёзными, если мы примем больного пациента за здорового, чем если мы примем здорового за больного. Нам важно охватить всех потенциально больных пациентов, чтобы сделать дополнительный анализ и удостовериться в результате.*

    Отлично, мы освежили в памяти ошибки I и II рода — это поможет нам понять суть метрик классификации. Давайте перейдём к ним.

### <center>МЕТРИКИ КЛАССИФИКАЦИИ

Будем рассматривать метрики для задачи классификации на следующем примере.

Мы случайным образом выбрали десять пациентов из нашей таблицы и моделью log_reg_full предсказали для них ответы:

![](./image/ML_3_1.png)

Все метрики, которые мы рассмотрим, основаны на матрице ошибок. С неё мы и начнём наш разбор.

1. **Матрица ошибок (confusion matrix)** *показывает все возможные исходы совпадения и несовпадения предсказания модели с действительностью. Используется для расчёта других метрик.*

Допустим, что у нас есть два класса и алгоритм, предсказывающий принадлежность каждого объекта к одному из классов. Тогда каждая ячейка матрицы ошибок соответствует количеству объектов, попавших в одну из следующих четырёх категорий:

* **Истинно положительные (True Positive, TP)** — это объекты, обозначенные моделью как класс 1 ($y^`=1$) и действительно принадлежащие к классу 1 ($y=1$).
* **Ложноположительные (False Positive, FP)** — это объекты, обозначенные моделью как класс 1 ($y^`=1$), но в действительности принадлежащие к классу 0 ($y=0$). То есть это объекты, для которых модель совершила ошибку I рода.
* **Истинно отрицательные (True Negative, TN)** — это объекты, обозначенные моделью как класс 0 ($y^`=0$) и действительно принадлежащие к классу 0 ($y=0$).
* **Ложноотрицательные (False Negative, FN)** — это объекты, обозначенные моделью как класс 0 ($y^`=0$), но в действительности принадлежащие к классу 1 ($y=1$). То есть это объекты, для которых модель совершила ошибку II рода.

Общий вид матрицы ошибок будет следующим:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_1.png)

Давайте посмотрим, как будет выглядеть матрица ошибок для нашего примера предсказаний модели *log_reg_full*:

![](./image/ML_3_2.png)

Для большей наглядности представим следующую ситуацию: у нас есть множество наблюдений двух классов (класс 1 — круги, класс 0 — квадраты). Пусть мы нарисовали некоторый круг и условились, что все объекты, лежащие в этом круге, мы будем считать классом 1, а объекты вне круга — классом 0. Тогда мы получим следующую картину:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_2.png)

Таким образом, мы получили четыре группы объектов:

* Объекты, попавшие в круг, для которых мы предсказали класс 1, и эти точки действительно относятся к классу 1 — *True Positive (TP)*.

* Объекты, попавшие в круг, для которых мы предсказали класс 1, но эти точки относятся к классу 0 — *False Positive (FP)*.

* Объекты за пределами круга, для которых мы предсказали класс 0, и эти точки действительно относятся к классу 0 — *True Negative (TN)*.

* Объекты за пределами круга, для которых мы предсказали класс 0, но эти точки относятся к классу 1 — *False Negative (FN)*.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_3.png)

Формально матрица ошибок не является метрикой, но на её основе составляются сами метрики классификации. Давайте перейдём к ним.

2. **Accuracy (достоверность/аккуратность)** *— доля правильных ответов модели среди всех ответов. Правильные ответы — это истинно положительные (True Positive) и истинно отрицательные ответы (True Negative):*

![](./image/ML_3_3.png)

**Примечание.** Нередко в русской литературе вы можете встретить перевод метрики *accuracy* как «точность», однако так же на русский язык переводится метрика *precision*, о которой мы поговорим далее. Поэтому, если вы используете термин «точность», старайтесь указывать, о какой именно метрике (*accuracy* или *precision*) идёт речь.

В виде диаграммы соотношение количества объектов, классы которых мы угадали и общего количества объектов записывается в следующем виде:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_4.png)

**Интерпретация:** *как много (в долях) модель угадала ответов.*

Метрика изменяется в диапазоне от 0 до 1. Чем ближе значение к 1, тем больше ответов модель «угадала». 

Рассчитаем accuracy для нашего примера:

![](./image/ML_3_4.png)

Итак, наша accuracy равна 0.5, то есть модель сделала верное предсказание для 50 % пациентов из выборки.

`Accuracy — самая простая и самая понятная метрика классификации, но у неё есть один существенный недостаток. Она бесполезна, если классы сильно несбалансированы. Продемонстрируем это следующим примером.`

    Допустим, мы хотим оценить работу спам-фильтра электронной почты. Спам-письма обозначены как Positive (1), а не спам — как Negative (0).

У нас есть 110 писем. Из них:

* 100 писем — не спам,
* 10 писем — спам.

Мы построили некоторый классификатор. Согласно результатам классификатора, из 100 писем класса «не спам» мы верно определили (отнесли к классу 0) 90, а оставшиеся 10 отнесли к классу «спам» (классу 1). То есть True Negative = 90, а False Positive = 10. Из 10 спам-писем мы верно определили 5 (отнесли к классу 1), а остальные 5 отнесли к классу «не спам» (классу 0). То есть True Positive = 5, а False Negative = 5.

Матрица ошибок будет иметь вид:

![](./image/ML_3_5.png)

Тогда *accuracy*:

![](./image/ML_3_6.png)

Однако представим, что мы построили классификатор, который просто предсказывает все письма как «не спам», то есть True Negative = 100, False Negative = 10, True Positive = 0, False Positive = 0.

Матрица ошибок будет иметь вид:

![](./image/ML_3_7.png)

Тогда *accuracy* будет равна:

![](./image/ML_3_8.png)

    Метрика выросла, однако наша «модель» не обладает никакой предсказательной силой, так как изначально мы хотели определять письма со спамом. Преодолеть это нам поможет переход с общей для всех классов метрики к отдельным показателям качества классов, а именно к метрикам precision, recall и -мера.

3. **Precision (точность), или PPV (Positive Predictive Value)** *— это доля объектов, названных классификатором положительными и при этом действительно являющихся таковыми, по отношению ко всем названным положительными объектам.*

![](./image/ML_3_9.png)

В виде диаграммы соотношение количества объектов класса 1, которые мы угадали и количества объектов, которые мы приняли за класс 1, записывается следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_5.png)

Метрика также изменяется от 0 до 1. 

**Интерпретация:** способность отделить класс 1 от класса 0. Чем больше *precision*, тем меньше ложных попаданий. **То есть чем ближе *precision* к 1, тем меньше вероятность модели допустить ошибку I рода.**

Именно *precision* не позволяет записать все ответы в один класс, так как в таком случае резко возрастает значение *False Positive* и метрика снижается.

Рассчитаем *precision* для нашего примера:

![](./image/ML_3_10.png)

Таким образом, количество названных моделью больных диабетом и при этом действительно являющихся больными составляет 67 % от всех пациентов.

Precision нужен в задачах, где от нас требуется минимум ложных срабатываний. Чем выше «цена» ложноположительного результата, тем выше должен быть *precision*.

Например, по камерам видеонаблюдения мы автоматически выявляем признаки драки на улицах и отправляем наряд полиции для урегулирования конфликта. Однако штат сотрудников сильно ограничен, реагировать на любой признак конфликта мы не можем, поэтому мы хотим уменьшить количество ложных вызовов. В такой ситуации мы выберем такую модель, у которой наибольший *precision*.

В предельном случае (когда *precision* равен 1) у модели отсутствуют ложноположительные срабатывания.

**Примечание.** *Важно понимать, что данный вывод справедлив только для выборки, на которой мы оцениваем метрику, то есть это не означает, что модель вовсе не может допустить ложноположительных результатов. Однако чем больше выборка, на которой мы тестируем алгоритм, тем ближе к истине будет данный вывод.*

4. **Recall (полнота), или TPR (True Positive Rate)** *— это доля объектов, названных классификатором положительными и при этом действительно являющихся таковыми, по отношению ко всем объектам положительного класса.*

![](./image/ML_3_11.png)

Диаграмма:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_3_6.png)

Метрика изменяется от 0 до 1.

**Интерпретация:** способность модели обнаруживать класс 1 вообще, то есть охват класса 1. Заметьте, что метрика зависит от количества ложноотрицательных срабатываний. **То есть чем ближе recall к 1, тем меньше вероятность модели допустить ошибку II рода.**

Рассчитаем *recall* для нашего примера:

![](./image/ML_3_12.png)

Итак, процент пациентов, которых модель определила к классу больных диабетом, среди всех действительно больных диабетом составляет 57 %.

*Recall* очень хорошо себя показывает в задачах, где важно найти как можно больше объектов, принадлежащих к классу 1.

Например, это различные задачи, в которых мы пытаемся определить наличие эффекта, который может привести к серьёзным последствиям. Это могут быть те же диагностические задачи или задачи, в которых мы прогнозируем вероятность отказа устройства, от работы которого зависят человеческие жизни.

Предельный случай (когда *recall* равен 1) означает, что модель нашла все объекты класса 1, например всех действительно больных пациентов. Однако метрика ничего не скажет о том, с какой точностью мы это сделали.

**Примечание.** *Важно понимать, что данный вывод справедлив только для выборки, на которой мы оцениваем метрику, то есть это не означает, что модель вовсе не может допустить ложноотрицательных исходов. Однако чем больше выборка, на которой мы тестируем алгоритм, тем данный вывод будет ближе к истине.*

Метрики *precision* и *recall* не зависят от сбалансированности классов и в совокупности дают довольно исчерпывающее представление о классификаторе. Однако на практике часто бывает так, **что увеличение одной из метрик может привести к уменьшению другой.**

Концентрация только на одной метрике (*precision* или *recall*) без учёта второй — сомнительная идея.

В битве за максимум *precision* для класса 1 побеждает модель, которая всегда будет говорить «нет». У неё вообще не будет ложноположительных срабатываний.

В битве за максимум *recall* для класса 1 побеждает модель, которая всегда будет говорить «да». Она охватит все наблюдения класса 1, и у неё не будет ложноотрицательных срабатываний. 

В реальности необходимо балансировать между двумя этими метриками.

    Классическим примером является задача определения оттока клиентов.

    Очевидно, что мы хотим найти как можно больше клиентов, которые потенциально могут уйти от нас. Чтобы повысить их лояльность, мы планируем использовать ресурсы колл-центра. Однако они ограничены и мы не можем звонить всем клиентам. Определив стратегию и ресурс для удержания клиентов, мы можем подобрать нужные пороги по precision и recall. Например, можно сосредоточиться на удержании только высокодоходных клиентов или тех, кто уйдёт с большей вероятностью. 

5. В таком случае нам подойдёт следующая метрика.

*$F_{betta}$ **(F-мера)** — это **взвешенное среднее гармоническое** между *precision* и *recall*:*

![](./image/ML_3_13.png)

*где $betta$ — это вес *precision* в метрике: чем больше $betta$, тем больше вклад.*

В частном случае, когда $betta=1$, мы получаем равный вклад для *precision* и *recall*, а формула будет выражать простое среднее гармоническое, или метрику $F_1$ ($F_1$-мера):

![](./image/ML_3_14.png)

Рассчитаем метрику $F_1$ для нашего примера:

![](./image/ML_3_15.png)

**В чём преимущество $F_1$-меры?**

*Метрика равна своему максимуму (1), если и precision, и recall равны 1 (то есть когда отсутствуют как ложноположительные, так и ложноотрицательные срабатывания). Но если хотя бы одна из метрик будет близка к 0, то и $F-мера$ будет близка к 0.*

Несмотря на отсутствие бизнес-интерпретации, метрика $F_1$ является довольно распространённой и используется в задачах, где необходимо выбрать модель, которая балансирует между precision и recall.

    Например, если цена дополнительной диагностики заболевания очень высока, то есть ложных срабатываний должно быть минимум, но при этом нам важно охватить как можно больше больных пациентов. 

**Примечание.** *Ещё одно небольшое, но очень важное замечание: все суждения, которые мы привели по отношению к precision, recall и $F$-мере, относятся только к классу 1, так как эти метрики по умолчанию считаются для класса 1. Для решения большинства задач знания о значении этих метрик для класса 1 более чем достаточно, так как обычно нас интересует именно наличие некоторого эффекта.*

*Однако если вам по каким-то причинам необходимо рассчитать precision, recall и $F$-меру для класса 0, для этого достаточно сделать перекодировку классов — поменять их обозначения местами или (при расчёте метрик с помощью библиотеки sklearn) изменить значение специального параметра pos_label на 0.*

Давайте обобщим всё вышесказанное в виде таблицы:

![](./image/ML_3_16.png)

### <center>РАСЧЁТ МЕТРИК НА PYTHON

Код и результаты находятся в `Логическая_регресия.ipynb/Метрики классификации`

### <center>ДОСТОИНСТВА И НЕДОСТАТКИ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ 

Давайте обобщим всё вышесказанное и приведём достоинства и недостатки логистической регрессии.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@happy-icon.png)

* Простой, интерпретируемый, но в то же время эффективный алгоритм. Благодаря этому он очень популярен в мире машинного обучения.
* Поиск параметров линейный или квадратичный (в зависимости от метода оптимизации), то есть ресурсозатратность алгоритма очень низкая.
* Не требует подбора внешних параметров (гиперпараметров), так как практически не зависит от них.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@sad-icon.png)

* Алгоритм работает хорошо, только когда классы линейно разделимы, что в реальных задачах бывает очень редко. Поэтому обычно данная модель используется в качестве baseline.

В завершение изучения логистической регрессии можно добавить, что недостаток с линейной разделимостью классов можно побороть с помощью введения полиномиальных признаков, тем самым снизив смещение модели. Тогда логистическая регрессия вместо разделяющей плоскости будет означать выгнутую разделяющую поверхность сложной структуры.

Однако мы знаем, что с этим трюком стоит быть аккуратным, так как можно получить переобученную модель. Поэтому в комбинации с полиномиальными признаками стоит подобрать наилучший параметр регуляризации.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-3_18.png)

На рисунке выше изображены три различные модели:

* первая — простая логистическая регрессия;
* вторая — логистическая регрессия, обученная на полиномиальных признаках второй степени;
* третья — логистическая регрессия, обученная на полиномиальных признаках десятой степени.

Видно, что первая модель обладает низким качеством и не обобщает общей зависимости (у неё высокое смещение). Третья же, напротив, идеально выделяет каждое наблюдение в правильный класс, но является переобученной и также не отражает общей зависимости (у неё высокий разброс). Оптимальной моделью является вторая, которая не подстраивается под индивидуальные наблюдения и отражает общую зависимость в данных.

    ✍ Ранее мы с вами рассмотрели основы бинарной классификации. Но что делать, когда классов, на которые необходимо разделить данные, больше 2? Например, классификация автомобилей по различным маркам или определение национальности по фотографии и т. д.

В таком случае используется очень простой подход, который называется **«один против всех» (one-vs-over)**.

**Идея** *этого подхода очень простая. Если у нас есть $k$ различных классов ($k>2$), давайте обучим $k$ классификаторов, каждый из которых будет предсказывать вероятности принадлежности каждого объекта к определённому классу.*

Например, у нас есть три класса, обозначенные как 0, 1 и 2. Тогда мы обучаем три классификатора: первый из них учится отличать класс 0 от классов 1 и 2, второй — класс 1 от классов 0 и 2, а третий — класс 2 от классов 1 и 0. Таким образом, класс, на который «заточен» классификатор, мы обозначаем как 1, а остальные классы — как 0.

Когда каждая из трёх моделей сделает предсказание вероятностей для объекта, итоговый классификатор будет выдавать класс, который соответствует самой «уверенной» модели.

Схематично это можно представить следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_4_1.png)

Если мы используем в качестве классификатора логистическую регрессию и количество факторов равно двум ($x_1$ и $x_2$), то можно изобразить тепловую карту вероятностей принадлежности к каждому из классов в каждой точке пространства, а также разделяющие плоскости, которые образуются при пороге вероятности в 0.5.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-3_10.png)

На тепловых картах каждый класс обозначен своим цветом: 0 — зелёным, 1 — жёлтым, 2 — синим. Чем ярче цвет, тем выше вероятность принадлежности к каждому к классу в этой области пространства.

В результате у нас получится три различных пространства вероятностей, что-то вроде трёх параллельных реальностей. Чтобы собрать всё это воедино, мы выбираем в каждой точке пространства максимум из вероятностей. Получим следующую картину:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-3_11.png)

Модель логистической регрессии легко обобщается на случай мультиклассовой классификации. Пусть мы построили несколько разделяющих плоскостей с различными наборами параметров $k$, где $k$ — номер классификатора. То есть имеем $K$ разделяющих плоскостей:

![](./image/ML_3_17.png)

Чтобы преобразовать результат каждой из построенных моделей в вероятности в логистической регрессии, используется функция softmax — многомерный аналог сигмоиды:

![](./image/ML_3_18.png)

Данная функция выдаёт нормированные вероятности, то есть в сумме для всех классов вероятность будет равна 1.

### <center>МУЛЬТИКЛАССОВАЯ КЛАССИФИКАЦИЯ НА PYTHON

Код и результаты находятся в `Мультиклассовая_классификация.ipynb`

    ✍ В этом юните мы познакомимся с ещё одним семейством моделей машинного обучения — деревьями решений. Для начала поговорим о том, что такое дерево решений и как с его помощью решают задачу классификации.

    Деревья решений являются одним из наиболее понятных человеку и в то же время мощных алгоритмов принятия решений. К тому же на их основе строятся самые эффективные ансамблевые модели машинного обучения, такие как случайный лес, о котором мы поговорим далее.

    Алгоритмы на основе деревьев решений могут использоваться как для решения задач классификации, так и для регрессии. В этом модуле мы разберём задачу классификации, а в дальнейшем, когда будем разбирать математическую составляющую алгоритмов, поговорим о том, как научить дерево решать задачу регрессии.

`Если коротко, решающее дерево предсказывает значение целевой переменной с помощью применения последовательности простых решающих правил. Этот процесс в некотором смысле согласуется с естественным для человека процессом принятия решений.`

### <center>ОБЩЕЕ ПРЕДСТАВЛЕНИЕ О ДЕРЕВЕ РЕШЕНИЙ

Начнём сразу с примера.

Представьте, что у вас есть автомобиль, который вы решили застраховать. Вы приходите в страховую компанию, где вам дают заполнить анкету. По этой анкете сотрудник страховой компании будет принимать решение, стоит ли выдавать вам страховку.

Сотрудник в свою очередь будет руководствоваться примерно следующим регламентом:

* Если возраст владельца > 40 лет, то:
  * Если место эксплуатации автомобиля — город, то:
    * Если стаж > 10 лет, то:
      * Застраховать.
    * Если стаж < 10 лет, то:
      * Не страховать.
  * Если место эксплуатации автомобиля — сельская местность, то:
    * Застраховать.
* Если возраст владельца ≤ 40 лет, то:
  * Если аварий не было зафиксировано, то:
    * Застраховать.
  * Если были аварии, то:
    * Если тип автомобиля — минивэн, то:
      * Застраховать.
    * Если тип автомобиля — спорткар, то:
      * Не страховать.

То есть сотрудник при принятии решения использует информацию, предоставленную вами в анкете, и подает её на вход вложенного условного оператора.

Для простоты восприятия можно представить такой подход визуально в виде следующего дерева:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_6_1.png)

Аналогичным образом работает и алгоритм машинного обучения под названием **«дерево решений» (Decision Tree)**. 

Если дерево уже обучено, то есть уже сформулированы условия в прямоугольниках, то, когда в страховую компанию придёт новый автовладелец, сотруднику будет достаточно прогнать данные клиента через дерево решений и таким образом принять решение, то есть произвести классификацию.

    Вот ещё один пример дерева решений. Большинство из нас когда-нибудь играли в игру «Слова на лбу» или «Тарантинки». На лоб каждого из игроков приклеивается бумажка с написанным на ней словом. Игрок может задавать другим игрокам вопросы о загаданном ему предмете/животном/человеке и т. д. Другие игроки могут отвечать на вопросы только «Да» и «Нет». Цель — за минимальное количество вопросов догадаться, о чём идёт речь.

Логика «если …, то …» используется людьми повседневно и поэтому интуитивно понятна каждому из нас. На основании этих рассуждений можно построить мощный алгоритм машинного обучения.

Деревья решений находят своё применение во множестве прикладных задач.

**Успешнее всего деревья применяют в следующих областях:**

* **Банковское дело.** Оценка кредитоспособности клиентов банка при выдаче кредитов.
* **Промышленность.** Контроль качества продукции (обнаружение дефектов в готовых товарах), испытания без нарушений (например, проверка качества сварки) и т. п.
* **Медицина.** Диагностика заболеваний разной сложности.
* **Молекулярная биология.** Анализ строения аминокислот.
* **Торговля.** Классификация клиентов и товара.

Теперь перейдём к формальной части. Нам важно уже сейчас познакомиться с терминологией деревьев решений, чтобы понять общий принцип их обучения.

Пусть у нас есть всё та же матрица наблюдений X, в которой содержатся наблюдения и характеризующие их признаки (привычный нам DataFrame), и правильные ответы y — метки классов. 

Дадим определение дереву решений и его составляющим ↓

Формально структура дерева решений — это **связный ациклический граф**. Что это значит?

**Граф** *— это абстрактная топологическая модель, которая состоит из вершин и соединяющих их рёбер.*

**Связный граф** *— это граф, в котором между любой парой существует направленная связь.*

**Ациклический граф** *— это граф, в котором отсутствуют циклы, то есть в графе не существует такого пути, по которому можно вернуться в начальную вершину.*

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_6_2.png)

**Примечание.** *Рекомендуем вам запомнить данное лаконичное определение дерева — так вы сможете показать свой уровень знаний перед будущим работодателем.*

В дереве решений можно выделить **три типа вершин:**

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_6_3.png)

* **Корневая вершина (root node)** — то, откуда всё начинается. Это первый и самый главный вопрос, который дерево задаёт объекту. В примере со страхованием это был вопрос «Возраст автовладельца > 40».
* **Внутренние вершины (intermediate nodes)** — это дополнительные уточняющие вопросы, которые дерево задаёт объекту. 
* **Листья (leafs)** — конечные вершины дерева. Это вершины, в которых содержится конечный «ответ» — класс объекта.

*Максимально возможная длина от корня до самых дальних листьев (не включая корневую) называется **максимальной глубиной дерева (max depth)**.*

Во внутренней или корневой вершине признак проверяется на некий логический критерий, по результатам которого мы движемся всё глубже по дереву. Например, «Количество кредитов $<=$ 1». 

*Логический критерий, который находится в каждой вершине, называется **предикатом**, или **решающим правилом**.*

На самом деле все предикаты — это просто взятие порога по значению какого-то признака. Формально это записывается следующим образом:

![](./image/ML_3_19.png)

Предикат вершины дерева $B_v$ (где $v$ — это номер вершины) равен 1 («Да»), если признак $x_j$ меньше либо равен значению $t$, и 0 («Нет») — в противном случае. Функция $I$ с квадратными скобками — это уже знакомая нам индикаторная функция: она равна 1, если условие внутри скобок выполняется, и 0 — в противном случае.

**Примечание.** *В зависимости от реализации предикат может быть с условием $<=$ или $>=$. В реализации sklearn используется условие $<=$. Но вы можете встретить другую формулировку предикатов в иных реализациях или в литературе.*

Если результат предиката равен 1, то мы переходим по левой ветви дерева к следующему узлу, в противном случае — по правой ветви дерева к следующему узлу.

**А что насчёт геометрии?**

Каждый новый вопрос дерева решений при его обучении разбивает пространство признаков на две части: в первую часть отправляются наблюдения, для которых предикат истинен, а во вторую — для которых он ложен.

_________

Посмотрим, как это будет выглядеть, на примере. 

Вам уже знакома задача классификации про ирисы. **Ирисы Фишера** — это задача, на которой Рональд Фишер ещё в 1936 году (почти 100 лет назад!) продемонстрировал работу алгоритма, разделяющего ирисы на сорта в зависимости от параметров долей околоцветника.

Пусть у нас есть следующие признаки:

* длина внутренней доли околоцветника (англ. *petal length*);
* ширина внутренней доли околоцветника (англ. *petal width*).

На основании этих двух признаков требуется разделить ирисы на три сорта:

* ирис щетинистый (*Iris Setosa*);
* ирис виргинский (*Iris virginica*);
* ирис разноцветный (*Iris versicolor*).

Пусть мы обучили на этих данных дерево решений с максимальной глубиной 2. Оно получилось вот таким:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-5_4.png)

В каждом блоке указаны следующие данные:

* Предикат $I[x_j<=t]$ — условие, по которому выборка делится на две части: на ту, для которой условие выполняется, и ту, для которой не выполняется.
* *gini* — критерий информативности Джини, о котором мы поговорим чуть позже.
* *samples* — количество объектов, которые мы проверяем на данном шаге.
* *value* — распределение по классам для объектов, которые мы проверяем на данном шаге: например `value=[0, 50, 50]` означает, что на текущем этапе разделения в выборке находится 0 объектов класса setosa и по 50 объектов классов *versicolor* и *virginica*.
* *class* — класс, который мы присваиваем, если завершим выполнение алгоритма на данном шаге.

А вот так будет выглядеть наш процесс разделения цветов на классы:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@dst3-ml3-5_5.png)

Как происходит построение разделяющих плоскостей?

* **Глубина дерева = 0.**

Дерево задаёт первый вопрос: $petal length <= 2.45$. Это выражение соответствует вертикальной прямой, которая делит пространство на две части по признаку *petal length*.

  * В левую часть пространства попали 50 наблюдений. Это только жёлтые точки пространства — цветы setosa. Значит, дальнейшее разделение не имеет смысла.

  * В правую часть пространства попали 100 наблюдений. Это и синие, и зелёные объекты классов *versicolor* и *virginica*. Значит, нужно попробовать задать ещё одно решающее правило.

* **Глубина дерева = 1.**

Дерево задаёт второй вопрос: $petal width <= 1.75$. Это выражение соответствует горизонтальной прямой, которая делит оставшееся после прошлого разделения пространство на две части по признаку *petal width*.

  * В нижнюю (синюю) часть этого пространства попали 54 наблюдения. Из них 49 цветов класса *versicolor* и 5 цветов класса *virginica*.

    Максимальная глубина достигнута. В полученной части пространства преобладает класс *versicolor*, значит все наблюдения, которые находятся в этой части, дерево будет относить к классу *versicolor*.

  * В верхнюю (зелёную) часть этого пространства попали 46 наблюдений. Из них 1 цветок класса *versicolor* и 45 цветов класса *virginica*.

    Максимальная глубина достигнута. В полученной части пространства преобладает класс *virginica*, значит все наблюдения, которые находятся в этой части, дерево будет относить к классу *virginica*.

Отметим, что деление пространства можно продолжать до тех пор, пока пространство не будет разделено так, чтобы верно выделить каждый из классов. 

Кстати, для каждой области можно подсчитать вероятность каждого из классов. Это просто отношение количества объектов $k$-класса, которые попали в лист дерева, к общему количеству объектов в листе.

Например, для синей области вероятности будут равны:

![](./image/ML_3_20.png)

_______

Теперь, когда мы разобрались с терминологией и геометрией, давайте поговорим о том, как строится решающее дерево.

### <center>ПРОЦЕСС ПОСТРОЕНИЯ ДЕРЕВА РЕШЕНИЙ

    ✍ Существует множество стратегий построения деревьев решений. Мы рассмотрим стратегию, реализованную в библиотеке sklearn, — алгоритм CART (Classification and Regression Tree), который предназначен для построения бинарных деревьев решений (деревьев, у которых каждая вершина связана с двумя другими вершинами нижнего уровня). Данный алгоритм, как следует из его названия, предназначен для решения задач классификации и регрессии.

Внимательный студент уже заметил, что построение дерева решений можно описать рекурсией. Каждая вершина дерева порождает две других вершины, а они в свою очередь порождают новые вершины, и так происходит до тех пор, пока не выполнится некоторый критерий остановки, например в вершине не останутся только наблюдения определённого класса.

**Примечание.** *Если вы забыли, что такое рекурсия, рекомендуем вам вернуться к модулю по продвинутому использованию функций и активировать рекурсивное мышление, оно нам понадобится.*

Пусть у нас есть матрица наблюдений X и столбец с ответами — метками классов y. На основе примеров и ответов мы хотим построить дерево решений, которое будет производить классификацию.

Итак, псевдокод рекурсивной функции для построения решающего дерева будет выглядеть следующим образом (запускать код не нужно, так как он является абстрактным):

```
def build_decision_tree(X, y):
    node = Node()
    if stopping_criterion(X, y) is True:
        node = create_leaf_with_prediction(y)
	return node
    else:
        X_left, y_left, X_right, y_right = best_split(X, y)
        node.left = build_decision_tree(X_left, y_left)
        node.right = build_decision_tree(X_right, y_right)
```

Разберёмся, как работает алгоритм:

1. Создать новую вершину *node*.

На первой итерации это будет корневая вершина. На последующих это будут внутренние вершины.

2. Проверить некоторый критерий остановки `stop_criterion()`.

Например, критерием остановки может быть следующее условие: все объекты, которые попали в вершину, — это объекты одного и того же класса.

Или достигнута максимальная глубина дерева (max_depth), например 5. Это значит, что дерево не будет продолжать делиться, если глубина уже равна 5.

Другой критерий: число наблюдений в листе (в sklearn этот параметр обозначен как min_samples_leaf) меньше заданного, например 7. Это значит, что при выполнении такого условия дерево продолжит делиться в том случае, если решающее правило выполняется как минимум для 7 наблюдений.

**2.1. Если условие остановки выполняется:**

Проверить, какой класс преобладает в текущей вершине. Превратить текущую вершину дерева в лист, где всем наблюдениям, которые попали в эту вершину, присвоить метку преобладающего класса.

Прекратить построение дерева, вернув из алгоритма полученный лист.

**2.2. Если условие остановки не выполняется:**

Среди всех возможных предикатов $B_v(x,t) = I[x_j<=t]$ найти такой, который обеспечивает разбиение выборки наилучшим образом.

То есть нужно найти такой признак $x_j$ и пороговое значение $t$, при которых достигается максимум некоторой информативности (существуют разные меры информативности, о них поговорим ниже). Назовём эту часть алгоритма некоторой абстрактной функцией `best_split()`.

Например, в нашем примере с ирисами это был предикат $Petal.Length <= 2.45$. Он обеспечил наилучшее разделение пространства на две части.

В результате разбиения будут созданы два набора данных:

* `X_left, y_left` (левый), для которого выполняется условие $x_j<=t$;
* `X_right, y_right` (правый), для которого условие не выполняется.

Создаются две новые вершины: левая и правая, в каждую из которых отправляется соответствующий набор данных.

То есть происходит рекурсивный вызов функции `build_decision_tree()`, и для каждой новой вершины алгоритм повторяется вновь с новым набором данных.

**Примечание.** *Вершина дерева node задаёт целое поддерево идущих за ним вершин, если такие имеются, а не только саму вершину.*

Центральный момент в построении дерева решений по обучающему набору данных — найти такой предикат $B_v(x,t) = I[x_j<=t]$, который обеспечит наилучшее разбиение выборки на классы. 

Как дерево определяет, какой вопрос нужно задать в каждой из вершин? 

Например, в задаче кредитного скоринга мы можем задавать множество различных вопросов в разной последовательности. Предикаты $B_0$ в первой вершине могут быть различными:

* возраст заёмщика $<=$ 25 лет,
* возраст заёмщика $<=$ 40 лет,
* размер кредита $<=$ 1000 $,
* наличие детей $<=$ 0.5 (если наличие детей — бинарный категориальный признак: 1 — есть дети, 0 — нет детей),
* и так далее.

Видно, что на место $x_j и $t$ можно подставить любой признак и порог соответственно.

Признак $x_j$ и его пороговое значение $t$ в каждой из вершин и есть внутренние параметры дерева решений, которые мы пытаемся отыскать. Это аналог коэффициентов уравнения линейной и логистической регрессий. 

### <center>ПОИСК ПАРАМЕТРОВ ДЕРЕВА РЕШЕНИЙ

    → Обратите внимание, что внутренние параметры дерева решений кардинально отличаются от линейных моделей.

В линейных моделях мы пытались найти такие коэффициенты в уравнениях, при которых наблюдался минимум функции потерь.

В деревьях же мы пытаемся выбрать такие признаки $x_j$ и их пороговые значения $t$, при которых произойдёт разделение набора на две части по какому-то критерию наилучшим образом. В нашем псевдокоде этот процесс организован в виде функции `best_split()`.

    → Важно понимать, что дерево решений — это топологический алгоритм, а не аналитический, то есть структуру дерева не получится описать в виде формулы, как те же линейные модели. Поэтому про стандартные методы оптимизации, такие как градиентный спуск или тем более метод наименьших квадратов, можно забыть. 

Чтобы интуитивно понять, как организуется поиск параметров, вспомним про игру «Слова на лбу».

    Пусть один человек загадывает знаменитость, а второй пытается отгадать, задавая только вопросы, на которые можно ответить «Да» или «Нет» (опустим варианты «не знаю» и «не могу сказать»).

    Какой вопрос отгадывающий задаст первым делом? Конечно, такой, который лучше всего уменьшит количество оставшихся вариантов.

    К примеру, вопрос «Это Анджелина Джоли?» в случае отрицательного ответа оставит более 7.5 миллиардов вариантов для дальнейшего перебора (строго говоря, поменьше, ведь не каждый человек — знаменитость, но всё равно немало), а вот вопрос «Это женщина?» отсечёт уже около половины знаменитостей.

    То есть, признак пол намного лучше разделяет выборку людей, чем признак это Анджелина Джоли, национальность — испанец или любит футбол.

    Интуитивно это соответствует уменьшению некоторой неопределённости, или, иначе говоря, повышению прироста информативности.

В случае «угадайки» знаменитостей критериев отбора может быть бесчисленное количество. Но когда мы работаем с набором данных, у нас ограниченное количество признаков и для них есть ограниченное количество порогов. Тогда мы можем полным перебором найти такую комбинацию $j$ и $t$, которая обеспечит наилучшее уменьшение неопределённости.

Неопределённость можно измерять различными способами, в деревьях решений для этого используются **энтропия Шеннона** и **критерий Джини**. Мы подробно обсудим их реализацию в модулях по математике.

### <center>ДЕРЕВЬЯ РЕШЕНИЙ В SKLEARN

_______
Подробный разбор в `./Деревья_решений.ipynb`
________

### <center>ДОСТОИНСТВА И НЕДОСТАТКИ ДЕРЕВЬЕВ РЕШЕНИЙ

Обобщим всё вышесказанное, выделив основные достоинства и недостатки деревьев решений.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@happy-icon.png)

* Дерево решений не требует нормализации/стандартизации данных.
* Наличие пропусков не оказывает существенного влияния на построение дерева.
* За счёт своей простоты модель деревьев решений интуитивно понятна и легко объяснима даже людям, не разбирающимся в методе.
* Приятный побочный эффект построения дерева решений — получение значимости признаков. Однако коэффициенты значимости целиком и полностью зависят от сложности дерева.

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@sad-icon.png)

* В силу дискретной топологической структуры дерево не дифференцируется по параметрам: стандартные алгоритмы поиска параметров, такие как градиентный спуск, не работают. Приходится использовать полный перебор.

**Примечание.** *Количество перебираемых вариантов можно сократить, используя методы динамического программирования (их изучение не входит в рамки нашего курса).*

* Так как метод является жадным, он долго обучается из-за полного перебора. Требует затрат больших вычислительных мощностей (по сравнению с другими алгоритмами). Особенно это ощутимо при большом количестве признаков на глубоких деревьях.
* Очень сильная склонность к переобучению. Необходим подбор внешних параметров: max_depth, min_sample_leaf и другие (о том, как организовать этот подбор, мы поговорим в отдельном модуле).
* Небольшое изменение в данных может заметно повлиять на структуру дерева.
* При работе с непрерывными числовыми признаками дерево делит их на категории и теряет информацию. Лучше всего дерево работает, если перевести числовые признаки в категориальные.

_____
    ✍ Одна модель хорошо, а много — лучше! С этой фразы мы с вами приоткрываем дверь в удивительный мир ансамблевых моделей.

**Ансамблевые модели** или просто **ансамбли (ensembles)** *— это метод машинного обучения, где несколько простых моделей (часто называемых «слабыми учениками») обучаются для решения одной и той же задачи и объединяются для получения лучших результатов.*

`Необходимость использования ансамблей может возникнуть тогда, когда вы уже нашли хорошую модель и никак больше не можете повысить её качество. В этом случае можно перейти к более продвинутому методу: использовать не одну модель (пусть и очень хорошую), а ансамбли моделей.`

Ансамбли — передовые алгоритмы для решения сложных задач машинного обучения. Сегодня они и нейронные сети являются главными конкурентами и дают наилучшие результаты, благодаря чему используются крупными компаниями в продакшене.

    → В этом модуле мы коснёмся основой идеи использования ансамблей на примере бэггинга и посмотрим на его реализацию для решения задач классификации в библиотеке sklearn, а в дальнейшем разберёмся, что за математическая «магия» стоит за ансамблевыми методами.

Говоря простыми словами, ансамбли — это объединение простых моделей в одного гиганта. Но объединять модели можно как угодно: например, взять тысячу разных логистических регрессий, а затем на их предсказаниях построить дерево решений, линейную регрессию или вообще нейронную сеть. А можно обучить сотню деревьев решений — построить целый лес, а для предсказания взять среднее. Вариаций объединения может быть сколько угодно.

Существует **три проверенных способа построения ансамблей:**

* **Бэггинг** — параллельно обучаем множество одинаковых моделей, а для предсказания берём среднее по предсказаниям каждой из моделей.
* **Бустинг** — последовательно обучаем множество одинаковых моделей, где каждая новая модель концентрируется на тех примерах, где предыдущая допустила ошибку.
* **Стекинг** — параллельно обучаем множество разных моделей, отправляем их результаты в финальную модель, и уже она принимает решение.

Об ансамблях типов бустинг и стекинг мы поговорим чуть позже в модуле, посвящённом продвинутым методам машинного обучения. В этом модуле разберёмся, что из себя представляет бэггинг и причём тут деревья решений.

### <center>БЭГГИНГ. ОСНОВНЫЕ ИДЕИ

    В 1906 г. в городе Плимут (Великобритания) на сельской ярмарке был проведён эксперимент. Фрэнсис Гальтон в качестве развлечения посетителей ярмарки предложил им на глаз оценить вес выставленного на всеобщее обозрение быка. За правильные ответы организаторы шоу обещали призы. В результате в голосовании приняли участие около 800 человек — как заядлых фермеров, так и людей, далёких от скотоводческих дел. Собрав после этой ярмарки все результаты, Гальтон высчитал среднее арифметическое значение для всей выборки — 1197 фунтов. Реальный же вес быка оказался 1198 фунтов. Каким-то непостижимым образом разношерстная публика дала ответ, максимально приближенный к реальному показателю. То есть ответ публики был точнее, чем ответ отдельно взятого эксперта, например мясника или скотовода.

По схожему принципу «голосования толпы» и работает бэггинг.

**Бэггинг (bagging)** *— это алгоритм построения ансамбля путём параллельного обучения множества независимых друг от друга моделей.*

В основе алгоритма лежит статистический метод, который называется **бутстрэпом (bootstrap)**. Идея бутстрэпа заключается в генерации $k$ выборок размера $n$ (бутстрэп-выборок) из исходного набора данных размера $m$ путём случайного выбора элементов с повторениями в каждом из наблюдений. 

Схематично работу метода можно представить следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_7_1.png)

Мы взяли исходный набор данных размером $m=12$ наблюдений и сгенерировали $k=3$ бутстрэп-выборки размером $n=8$ наблюдений. Обратите внимание, что наблюдения в выборках могут повторяться.

В частном случае можно генерировать выборки того же размера, то есть $n=m$. Таким образом, мы из набора данных создаём $k$ датасетов из исходного того же размера.

**Примечание.** *Такие бутстрэп-выборки часто используются для оценки различных статистических показателей (например, разброса или доверительного интервала). Если вычислять статистические оценки на нескольких независимых выборках, то мы можем оценить их разброс. Поиск большого количества независимых выборок сложен в силу того, что для этого требуется слишком много данных. Поэтому мы используем бутстрэп, чтобы создать несколько выборок.*

Давайте обучим $k$ одинаковых моделей на каждой из сгенерированных выборок, сделаем предсказания, а затем усредним их. Так мы получим бэггинг.

Схематично такой подход можно описать следующим образом:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_7_2.png)

**Примечание.** *В случае классификации «усреднение» означает мажоритарное голосование (принцип большинства голосов). То есть объект относится к тому классу, за который проголосовало большинство алгоритмов.*

Важно отметить, что в бэггинге в голосовании принимает участие модель одного вида. Эта модель называется **базовой моделью (base model)**. Нельзя обучить на половине сгенерированных наборов данных логистические регрессии, а на второй половине — деревья решений.

Когда мы будем разбирать математическое обоснование ансамблей, мы докажем следующие два утверждения:

* Смещение (*bias*) бэггинг-ансамбля не больше ($<=$) смещения одного алгоритма из этого ансамбля.
* Однако разброс (*variance*) бэггинг-ансамбля в $k$ раз меньше, чем разброс одного алгоритма из ансамбля, где $k$ — количество алгоритмов в ансамбле.

**Что это значит в переводе с математического?**

*Теорема гарантирует, что средняя ошибка ансамбля, построенного по принципу бэггинга, не выше, чем средняя ошибка базовой модели, но при этом шанс переобучения алгоритма значительно ниже.*

Это очень важно для моделей, склонных к переобучению, таких как глубокие деревья решений.

### <center>СЛУЧАЙНЫЙ ЛЕС

**Случайный лес (Random Forest)** *— это самая распространённая реализация бэггинга, основанная на использовании в качестве базовой модели дерева решений.*

Помимо бутстрэпа, случайный лес использует метод случайных подпространств. Суть этого метода заключается в том, что каждая модель обучается не на всех признаках, а только на части из них. Такой подход позволяет уменьшить коррелированность между ответами деревьев и сделать их независимыми друг от друга.

<H_3>Алгоритм построения случайного леса для задачи классификации<H_3>

Пусть количество объектов в наборе данных равно N, а количество признаков — *M*. То есть размер набора данных — (*N, M*). Количество деревьев в лесу равно *K*. Тогда для обучения случайного леса необходимо выполнить следующие шаги:

1. С помощью бутстрэпа создать K наборов данных размера (*N, M*).
2. Для каждого сгенерированного набора данных применить метод случайных подпространств: выбрать *L < M* случайных признаков и получить K новых наборов данных размером (*N, L*).
3. На каждом наборе данных обучить дерево решений.

Когда поступят новые данные, нам нужно будет прогнать их через каждое дерево и объединить результаты отдельных деревьев мажоритарным голосованием или путём комбинирования вероятностей.

Ниже приведена схема работы описанного алгоритма для решения задачи классификации. Для простоты лес состоит из четырёх деревьев (*K=4*).

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_7_3.png)

_________
Давайте посмотрим, как работает алгоритм случайного леса, на примере. 

Пусть у нас есть набор данных со следующими факторами:

* $x_1$ — возраст,
* $x_2$ — доход в тысячах рублей,
* $x_3$ — группа крови.

**Целевой признак** ($y$) — подвергался ли человек операции хотя бы раз в жизни (1 — да, 0 — нет).

Тогда, если мы обучим алгоритм случайного леса на представленных данных, мы получим следующую картину:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_7_4.png)

Количество объектов в наборе данных N=5, а количество факторов — M=3 (+1 целевой признак, его мы не включаем в размерность). Таким образом, размер таблицы наблюдений — (5, 3). Количество деревьев в лесу K=4.

1. На первом этапе алгоритма мы формируем K=4 бутстрэп выборки размером (5, 3), выбирая из таблицы строки случайным образом с возможностью повторения.
2. На втором этапе мы случайным образом выбираем L=2 признаков из каждой таблицы и получаем четыре выборки размером (5, 2).
3. На третьем этапе мы обучаем K=4 деревьев решений, каждое на своей выборке.

*Обратите внимание, что деревья, из которых состоит лес, могут быть различной глубины и структуры в зависимости от того, насколько просто была разделима поданная выборка.*

Теперь, когда поступят новые данные, нам останется только подать их на вход каждого из деревьев, получить предсказания, а затем усреднить их путём мажоритарного голосования и получить ответ:

![](./image/asset-v1_SkillFactory+DST-3.0+28FEB2021+type@asset+block@ML_3_7_5.png)
_______

______
### <center>СЛУЧАЙНЫЙ ЛЕС В SKLEARN

Подробный разбор в `Введение_в_ансамбли_бэггинг._Случайный_лес.ipynb`
_______